## Testing the accuracy and efficiency of interpolation via xarray

### Add paths to run locally or off Cheyenne or Casper 
(what is the difference between these two again?)

In [1]:
import sys
# Add common resources folder to path
sys.path.append('/glade/u/home/jonahshaw/Scripts/git_repos/CESM2_analysis')
sys.path.append('/glade/u/home/jonahshaw/Scripts/git_repos/CESM2_analysis/Common/')
# sys.path.append("/home/jonahks/git_repos/netcdf_analysis/Common/")

from imports import (
    pd, np, xr, mpl, plt, sns, os, 
    datetime, sys, crt, gridspec,
    ccrs, metrics, Iterable
    )

from functions import (
    masked_average, add_weights, sp_map,
    season_mean, get_dpm, leap_year, share_ylims,
    to_png
    )

from cloud_metric import Cloud_Metric
from collections import deque
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Check running location and adjust working directory appropriately.

In [3]:
host = os.uname()[1]
if 'jupyter' in host.split('-'): # Check if running on NIRD through the Jupyter Hub
    print('Running through MC2 Jupyter Hub')
    model_dir = '/mnt/mcc-ns9600k/jonahks/'
    os.chdir(model_dir)
    
elif 'casper' in host: # Check if running on NIRD through the Jupyter Hub
    print('Running through CISL Casper')
    obs_dir = '/glade/u/home/jonahshaw/w/obs/'
#     model_dir = '/glade/p/cesm/pcwg/jenkay/COSP/'
    os.chdir(obs_dir)
    
elif 'r6i6' in host: # Check if running on NIRD through the Jupyter Hub
    print('Running through CISL Cheyenne')
    obs_dir = '/glade/u/home/jonahshaw/w/obs/'
#     model_dir = '/glade/p/cesm/pcwg/jenkay/COSP/'
    os.chdir(obs_dir)

else:  # Assume that we're running on a local machine and mounting NIRD
    print('Running on %s, attempting to mount ns9600k/jonahks/ from NIRD' % str(host))
    os.system('fusermount -zu ~/drivemount/')  # unmount first
    os.system('sshfs jonahks@login.nird.sigma2.no:"p/jonahks/" ~/drivemount/')    # Calling mountnird from .bashrc doesn't work
    os.chdir('/home/jonahks/drivemount/')
    save_dir = '~/DATAOUT/'
    save_to = os.path.expanduser(save_dir)

output_dir = '/glade/u/home/jonahshaw/figures/'
obs_dir = '/glade/u/home/jonahshaw/w/obs/'
model_dir = '/glade/p/cesm/pcwg/jenkay/COSP/'
# case_dir = '/glade/p/cesm/pcwg/jenkay/COSP/cesm21/' 

# Check that each important directory can be accessed:    
access_paths = os.path.exists(output_dir) and os.path.exists(model_dir) #and os.path.exists(conv_dir)
print('Can access all directory paths:', access_paths)

Running through CISL Casper
Can access all directory paths: True


## For Cheyenne from ATOC python forum. Doesn't work here (Casper?)

This was from a section by Eric Keenan, so I could think about reaching out to him. He's my ATOC mentor too, I think.  

https://colab.research.google.com/drive/1B4HzAK0a7-d6hEsNjlCIHvOpcfhlKBNH?usp=sharing#scrollTo=KS2xmSyct868

In [4]:
# CESM1 = xr.open_dataset('/content/drive/MyDrive/Non-academic/ATOC_Forum/ts_Amon_CESM1-CAM5_historical_rcp85_r1i1p1_185001-210012.nc', chunks={'time':10})
# http://xarray.pydata.org/en/stable/dask.html

#if using dask on Cheyenne, run this before executing your code:
import dask
from dask_jobqueue import PBSCluster

cluster = PBSCluster(cores=36, processes=8, memory="50GB",
                     project='ACCOUNT',  queue='regular',
                     resource_spec='select=1:ncpus=8:mem=50G',
                     walltime='01:00:00')
cluster.scale(18)

from dask.distributed import Client
client = Client(cluster)
client

#you can also bring up the dask dashboard as below

## Something that works!

In [6]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=8,
                     processes=16, memory="50GB",
                     project='UCUC0007',
                     queue='dav',
                     resource_spec='select=1:ncpus=4:mem=50G',
                     walltime='00:15:00')

cluster.scale(18)

from dask.distributed import Client
client = Client(cluster)
client


Client Scheduler: tcp://10.12.205.13:39782 Dashboard: http://10.12.205.13:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## To do later in dask (while testing efficiency?)

In [ ]:
# Break up into many tasks

@dask.delayed
def load(filename):
    ...

@dask.delayed
def process(filename):
    ...

@dask.delayed
def save(filename):
    ...


def f(filenames):
    results = []
    for filename in filenames:
        data = load(filename)
        data = process(data)
        result = save(data)

    return results

dask.compute(f(filenames))